# Generative Models and Dimensionality Reduction

This lecture is mostly about generative models, but I'll introduce a particular type of generative model that will help us see another aspect of dimensionality reduction along the way.

As physicists, when we think about data we tend to think about information that an experiment collects, such as momenta of electrons scattering off of one another, or the mass of a person falling into a large black hole. We tend to think of the physics of the objects themselves.

However, when we move to a computer it is more natural to think of data in terms of its representation on our computers. In that situation, one might think of a data space $M$, which could be discrete, or continuous, like for a manifold. This way of thinking has the advantage that it is inclusive of mathematical data, such as those associated to sharp mathematical definitions rather than numbers collected at an experiment.

In either case, whether its our experimental biases or our mathematical biases, we normally don't have data points for all of the potential data space $M$. Rather, our experiments or mathematical biases lead us to sample / collect data from a data distribution $\rho(M)$. Often, the data space $M$ lives in an ambient space $A$ that is (much) bigger, and it's easy to sample from $A$, but not from $M$.

Generative models are primarily about two things: 
1. learning to model some distribution or density $\rho(M)$ (which might be even more general than a "data distribution").
2. learning to sample from that distribution.

The first part is called density estimation, and the second part is called sampling.

# Autoencoder

Before we get to a generative model, we'll begin with the humble autoencoder, which is useful for dimensionality reduction.

We actually studied dimensionality reduction already, via principal component analysis (PCA). PCA is a linear dimensionality reduction technique, which means that it is a linear map $L: \mathbb{R}^n \to \mathbb{R}^m$ with $m < n$, where $m$ is the number of principal components. The map is chosen to maximize the variance of the data in the new space, and effectively compresses the information of the vectors in $\mathbb{R}^n$.

The autoencoder is a *nonlinear* dimensionality reduction technique. It is a neural network with a bottleneck layer, which is a layer with fewer neurons than the input or output layers. The network is trained to minimize the reconstruction error, which is the difference between the input and output of the network. The bottleneck layer forces the network to learn a compressed representation of the data, which is the output of the bottleneck layer.

For simplicity, we'll start with an autoencoder on a dataset called MNIST.

Let's show some data.

Now let's define the autoencoder. The forward pass of the NN, associated to the `forward` method, is the composition of an encoder NN and a decoder NN. The encoder compresses the data, and the decoder decompresses it to something of the same dimension as the input. For simplicity we'll just use fully connected networks for the encoder and decoder.

We take the loss function to be the binary cross-entropy loss on the pixels, which have been normalized to be between $0$ and $1$, which is enforced in the decoder by the sigmoid function.

This loss is the so-called "reconstruction loss" of the autoencoder, which measures how well it reconstructs the input after passing it through a bottleneck.

To train the autoencoder, we simply run the train loop across batches of data and backprop the reconstruction loss.

Let's create our autoencoder.

Let's compare the true images and the reconstructed image prior to training

Yes, it looks terrible, as it should. Let's see what it looks like after epoch (which we remind you, passes through *all* of the data).

This already looks quite good! Let's see what it looks like after 20 epochs.

This looks quite good, and we notice that the edges are fairly sharp.

In summary, the autoencoder has learned a compressed representation of the data, which is the output of the bottleneck layer. This compressed representation is a nonlinear dimensionality reduction of the data.

With this trained neural network you could *try* to generate new images by choosing points from the latent / bottleneck layer and passing it through the decoder. In general though, nothing in the problem ensures that the latent space is densely populated by the data, so this is not guaranteed to work. I encourage you to play around with this and try it, though!

# Variational Autoencoder

In contrast to an autoencoder, a variational autoencoder (VAE) is a generative model. Instead of encoding the input $x$ in a latent $z$, it actually models a distribution on the latent space. This is the distribution we wish to sample from to generate new data.

We will study a standard case where the distributionson the latents are normal, with a mean and variance that depend on $x$.

$z \sim \mathcal{N}(\mu(x), \sigma(x))$.

However, if we have stochastic later in the middle of the network, we can't backpropagate through it. To get around this, we use the reparameterization trick. We sample from a standard normal 

$\epsilon \sim \mathcal{N}(0, 1)$

 and then define 

$z = \mu(x) + \sigma(x) \epsilon$. 
 
This way, we can backpropagate through the encoder and decoder by taking the encoder of the AE and adding two extra layers that help it learn the mean and variance; this is the encoder of the VAE. Then, samples from the latent space are generated by sampling $\epsilon$ and passing it through the decoder.

Training proceeds via a loss with two components, the reconstruction loss and the KL divergence between the true distribution and the model distribution. In this case, we will use the original treatment of Kingma and Welling [https://arxiv.org/abs/1312.6114](https://arxiv.org/abs/1312.6114), that chooses a particular KL estimator suitable training the latent distribution to model the Gaussian.